In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from utils import *
from keras.models import Model, load_model, Sequential
from keras.layers import Dense,Flatten, Dropout, Conv2D, BatchNormalization, Activation, MaxPooling2D, concatenate
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint

2022-12-22 23:21:52.946963: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 23:21:53.127086: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-22 23:21:53.905211: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-12-22 23:21:53.905321: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer

In [128]:
from utils import *

# Create the training dataset

In [13]:
slide_nums = ['078','081', '091', '094', '096']
slide_paths = []
tumor_mask_paths = []
slide_windows1 = []
slide_windows2 = []
window_labels = []
for slide_num in slide_nums:
  slide_paths.append('tumor_' + slide_num + '.tif')
  tumor_mask_paths.append('tumor_' + slide_num + '_mask.tif')
for i in range(len(slide_paths)):
    window_1, window_2, labels = get_windows(slide_paths[i], tumor_mask_paths[i], (2,3), 
                                             stride=150, window_len=299)
    slide_windows1.extend(window_1)
    slide_windows2.extend(window_2)
    window_labels.extend(labels)


    

In [17]:
labels = np.array(window_labels)

# Create balanced dataset 

In [19]:
w_tumor, w_tumor2, wo_tumor, wo_tumor2 = shuffle_data(slide_windows1, slide_windows2, window_labels)

# Split dataset into 80:20

In [25]:
label_pos = np.ones(len(w_tumor))
label_neg = np.zeros(len(wo_tumor))
assert len(label_pos)==len(label_pos)

In [26]:
images1 = np.concatenate((w_tumor,wo_tumor), axis=0)
images2 = np.concatenate((w_tumor2,wo_tumor2), axis=0)
labels = np.concatenate((label_pos, label_neg))

In [28]:

shuffle = np.arange(len(labels))
np.random.shuffle(shuffle)

ds_images1 = np.array([images1[i] for i in shuffle])
ds_images2 = np.array([images2[i] for i in shuffle])
ds_labels = np.array([labels[i] for i in shuffle])
# ds_labels2 = np.array([labels[i] for i in shuffle])

In [30]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(ds_images1, ds_labels, test_size=0.2, shuffle=False)
X_train2, X_test2, y_train2, y_test2 = train_test_split(ds_images2, ds_labels, test_size=0.2, shuffle=False)

# Save the numpy array

In [31]:
with open('train1.npy', 'wb') as f:
    np.save(f, X_train1)
    np.save(f, y_train1)

with open('test1.npy', 'wb') as f:
    np.save(f, X_test1)
    np.save(f, y_test1)

In [32]:
with open('train2.npy', 'wb') as f:
    np.save(f, X_train2)
    np.save(f, y_train2)

with open('test2.npy', 'wb') as f:
    np.save(f, X_test2)
    np.save(f, y_test2)

# Load the numpy array

In [5]:
with open('train1.npy', 'rb') as f:
    X_train1 = np.load(f)
    y_train1 = np.load(f)

In [6]:
with open('train2.npy', 'rb') as f:
    X_train2 = np.load(f)
    y_train2 = np.load(f)

In [7]:
with open('test1.npy', 'rb') as f:
    X_test1 = np.load(f)
    y_test1 = np.load(f)

In [8]:
with open('test2.npy', 'rb') as f:
    X_test2 = np.load(f)
    y_test2 = np.load(f)

# Reduce the size of training dataset due to computing power

In [13]:
X_train1 = preprocess(X_train1)
X_train2 = preprocess(X_train2)
X_test1 = preprocess(X_test1)
X_test2 = preprocess(X_test2)

In [14]:
arr = np.concatenate([X_train1, X_test1])

In [16]:
arr2 = np.concatenate([X_train2, X_test2])

In [17]:
label = np.concatenate([y_train1, y_test1])

In [19]:
w_tumor, w_tumor2, wo_tumor, wo_tumor2 = cut_data(arr, arr2, label, size = 1250)


In [20]:
del arr, arr2, label

# Build the training and test dataset with reduced size of overall dataset

In [22]:
label_pos = np.ones(len(w_tumor))
label_neg = np.zeros(len(wo_tumor))
assert len(label_pos)==len(label_pos)

images1 = np.concatenate((w_tumor,wo_tumor), axis=0)
images2 = np.concatenate((w_tumor2,wo_tumor2), axis=0)
labels = np.concatenate((label_pos, label_neg))

len(labels)


shuffle = np.arange(len(labels))
np.random.shuffle(shuffle)

ds_images1 = np.array([images1[i] for i in shuffle])
ds_images2 = np.array([images2[i] for i in shuffle])
ds_labels = np.array([labels[i] for i in shuffle])
# ds_labels2 = np.array([labels[i] for i in shuffle])

ds_images1.shape

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(ds_images1, ds_labels, test_size=0.2, shuffle=False)
X_train2, X_test2, y_train2, y_test2 = train_test_split(ds_images2, ds_labels, test_size=0.2, shuffle=False)

# Save the dataset into numpy file

In [23]:
with open('train1_s.npy', 'wb') as f:
    np.save(f, X_train1)
    np.save(f, y_train1)

with open('test1_s.npy', 'wb') as f:
    np.save(f, X_test1)
    np.save(f, y_test1)

with open('train2_s.npy', 'wb') as f:
    np.save(f, X_train2)
    np.save(f, y_train2)

with open('test2_s.npy', 'wb') as f:
    np.save(f, X_test2)
    np.save(f, y_test2)

# Load the numpy array from numpy file and convert to tensor batch data

In [4]:
with open('train1_s.npy', 'rb') as f:
    X_train1 = np.load(f)
    y_train1 = np.load(f)

with open('train2_s.npy', 'rb') as f:
    X_train2 = np.load(f)
    y_train2 = np.load(f)

with open('test1_s.npy', 'rb') as f:
    X_test1 = np.load(f)
    y_test1 = np.load(f)

with open('test2_s.npy', 'rb') as f:
    X_test2 = np.load(f)
    y_test2 = np.load(f)

In [5]:
train_ds = create_tf_dataset(X_train1, X_train2, y_train2)

2022-12-22 21:43:56.324205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-22 21:43:56.337107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-22 21:43:56.338944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-22 21:43:56.341481: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
test_ds = create_tf_dataset(X_test1, X_test2, y_test2)

# Create Inception V3 model

In [9]:

model = create_model() 
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2input_2_2 (InputLayer)  [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                             

 activation_3 (Activation)      (None, 73, 73, 80)   0           ['batch_normalization_3[0][0]']  
                                                                                                  
 activation_97activation_97_2 (  (None, 73, 73, 80)  0           ['batch_normalization_97batch_nor
 Activation)                                                     malization_97_2[0][0]']          
                                                                                                  
 conv2d_4 (Conv2D)              (None, 71, 71, 192)  138240      ['activation_3[0][0]']           
                                                                                                  
 conv2d_98conv2d_98_2 (Conv2D)  (None, 71, 71, 192)  138240      ['activation_97activation_97_2[0]
                                                                 [0]']                            
                                                                                                  
 batch_nor

                                                                                                  
 conv2d_5 (Conv2D)              (None, 35, 35, 64)   12288       ['max_pooling2d_1[0][0]']        
                                                                                                  
 conv2d_7 (Conv2D)              (None, 35, 35, 64)   76800       ['activation_6[0][0]']           
                                                                                                  
 conv2d_10 (Conv2D)             (None, 35, 35, 96)   82944       ['activation_9[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 35, 35, 32)   6144        ['average_pooling2d[0][0]']      
                                                                                                  
 conv2d_99conv2d_99_2 (Conv2D)  (None, 35, 35, 64)   12288       ['max_pooling2d_5max_pooling2d_5_
          

 conv2d_15 (Conv2D)             (None, 35, 35, 64)   16384       ['mixed0[0][0]']                 
                                                                                                  
 conv2d_109conv2d_109_2 (Conv2D  (None, 35, 35, 64)  16384       ['mixed0mixed0_2[0][0]']         
 )                                                                                                
                                                                                                  
 batch_normalization_15 (BatchN  (None, 35, 35, 64)  192         ['conv2d_15[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_109batch_n  (None, 35, 35, 64)  192         ['conv2d_109conv2d_109_2[0][0]'] 
 ormalization_109_2 (BatchNorma                                                                   
 lization)

                                                                                                  
 batch_normalization_17 (BatchN  (None, 35, 35, 96)  288         ['conv2d_17[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_18 (BatchN  (None, 35, 35, 64)  192         ['conv2d_18[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_106batch_n  (None, 35, 35, 64)  192         ['conv2d_106conv2d_106_2[0][0]'] 
 ormalization_106_2 (BatchNorma                                                                   
 lization)                                                                                        
          

...

                                                                                                  
 conv2d_80 (Conv2D)             (None, 8, 8, 448)    573440      ['mixed8[0][0]']                 
                                                                                                  
 conv2d_174conv2d_174_2 (Conv2D  (None, 8, 8, 448)   573440      ['mixed8mixed8_2[0][0]']         
 )                                                                                                
                                                                                                  
 batch_normalization_80 (BatchN  (None, 8, 8, 448)   1344        ['conv2d_80[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_174batch_n  (None, 8, 8, 448)   1344        ['conv2d_174conv2d_174_2[0][0]'] 
 ormalizat

                                                                                                  
 batch_normalization_79 (BatchN  (None, 8, 8, 384)   1152        ['conv2d_79[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_82 (BatchN  (None, 8, 8, 384)   1152        ['conv2d_82[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_83 (BatchN  (None, 8, 8, 384)   1152        ['conv2d_83[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_84

                                                                  'activation_173activation_173_2[
                                                                 0][0]']                          
                                                                                                  
 concatenate_2concatenate_2_2 (  (None, 8, 8, 768)   0           ['activation_176activation_176_2[
 Concatenate)                                                    0][0]',                          
                                                                  'activation_177activation_177_2[
                                                                 0][0]']                          
                                                                                                  
 activation_178activation_178_2  (None, 8, 8, 192)   0           ['batch_normalization_178batch_no
  (Activation)                                                   rmalization_178_2[0][0]']        
          

 average_pooling2d_8 (AveragePo  (None, 8, 8, 2048)  0           ['mixed9[0][0]']                 
 oling2D)                                                                                         
                                                                                                  
 conv2d_181conv2d_181_2 (Conv2D  (None, 8, 8, 384)   442368      ['activation_180activation_180_2[
 )                                                               0][0]']                          
                                                                                                  
 conv2d_182conv2d_182_2 (Conv2D  (None, 8, 8, 384)   442368      ['activation_180activation_180_2[
 )                                                               0][0]']                          
                                                                                                  
 conv2d_185conv2d_185_2 (Conv2D  (None, 8, 8, 384)   442368      ['activation_184activation_184_2[
 )        

                                                                                                  
 activation_186activation_186_2  (None, 8, 8, 384)   0           ['batch_normalization_186batch_no
  (Activation)                                                   rmalization_186_2[0][0]']        
                                                                                                  
 batch_normalization_187batch_n  (None, 8, 8, 192)   576         ['conv2d_187conv2d_187_2[0][0]'] 
 ormalization_187_2 (BatchNorma                                                                   
 lization)                                                                                        
                                                                                                  
 activation_85 (Activation)     (None, 8, 8, 320)    0           ['batch_normalization_85[0][0]'] 
                                                                                                  
 mixed9_1 

In [10]:
BATCH_SIZE = 16
model.compile(optimizer=RMSprop(learning_rate = 0.002,
                                momentum=0.9,
                                decay = 0.9,
                                epsilon = 1.0
                               ),
              loss='binary_crossentropy',
              metrics=['accuracy'])


history = model.fit(
    train_ds,
    steps_per_epoch=int(len(X_train1)/BATCH_SIZE)+1,
    epochs=10,
    validation_data=test_ds,
    validation_steps=int(len(X_test1)/BATCH_SIZE)+1,)

Epoch 1/10


2022-12-22 21:46:47.862442: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8200


126/126 [==============================] - 93s 574ms/step - loss: 0.5457 - accuracy: 0.7599 - val_loss: 0.4174 - val_accuracy: 0.8379
Epoch 2/10
126/126 [==============================] - 69s 546ms/step - loss: 0.3289 - accuracy: 0.8606 - val_loss: 0.3385 - val_accuracy: 0.8867
Epoch 3/10
126/126 [==============================] - 68s 544ms/step - loss: 0.2967 - accuracy: 0.8800 - val_loss: 0.3240 - val_accuracy: 0.8730
Epoch 4/10
126/126 [==============================] - 68s 544ms/step - loss: 0.2732 - accuracy: 0.8894 - val_loss: 0.3173 - val_accuracy: 0.8672
Epoch 5/10
126/126 [==============================] - 69s 545ms/step - loss: 0.2582 - accuracy: 0.8953 - val_loss: 0.3115 - val_accuracy: 0.8711
Epoch 6/10
126/126 [==============================] - 68s 543ms/step - loss: 0.2470 - accuracy: 0.9058 - val_loss: 0.3075 - val_accuracy: 0.8730
Epoch 7/10
126/126 [==============================] - 68s 543ms/step - loss: 0.2379 - accuracy: 0.9092 - val_loss: 0.3040 - val_accuracy: 0.8

# Save the model 

In [11]:
del X_train1, X_train2, X_test1, X_test2, y_train1, y_train2, y_test1, y_test2

In [13]:
model_filename = os.path.join('./model', "inception_rmsprop")
model.save(model_filename)

INFO:tensorflow:Assets written to: ./model/inception_rmsprop/assets


INFO:tensorflow:Assets written to: ./model/inception_rmsprop/assets
